# Week 2

In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizerFast, DataCollatorWithPadding

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_json("Sarcasm_Headlines_Dataset_v2.json", lines=True)

In [3]:
texts = df["headline"].tolist() # data
labels = df["is_sarcastic"].tolist() # class
# covert pandas series to python list for train, test, split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(texts, labels, train_size=0.8, stratify=labels, random_state=42)
# 80/20 split with seed 42

In [5]:
print(len(X_train), len(X_test))

22895 5724


In [6]:
tokenizer = DistilBertTokenizerFast.from_pretrained(
    "distilbert-base-uncased"
)

In [7]:
train_encodings = tokenizer(
        X_train,
        truncation=True,
        padding=False,
        max_length=128
    )
    
test_encodings  = tokenizer(
        X_test,
        truncation=True,
        padding=False,
        max_length=128
    )

In [8]:
train_samples = [
    {
        "input_ids": train_encodings["input_ids"][i],
        "attention_mask": train_encodings["attention_mask"][i],
        "labels": y_train[i]
    }
    for i in range(len(y_train))
]

test_samples = [
    {
        "input_ids": test_encodings["input_ids"][i],
        "attention_mask": test_encodings["attention_mask"][i],
        "labels": y_test[i]
    }
    for i in range(len(y_test))
]

In [9]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_loader = DataLoader(
    train_samples,
    batch_size=16,
    shuffle=True,
    collate_fn=data_collator
)

test_loader = DataLoader(
    test_samples,
    batch_size=16,
    shuffle=False,
    collate_fn=data_collator
)

# Week 3

In [10]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=100,
    save_total_limit=1
)

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [18]:
from transformers import Trainer # Trainer class

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_samples,
    eval_dataset=test_samples,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/var/folders/fw/bh05vf51245dmqm8wmvz3k_00000gn/T/ipykernel_14716/127083574.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### model training

In [14]:
trainer.train()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
100,0.499300
200,0.373000
300,0.288700
400,0.296500
500,0.278500
600,0.254900
700,0.273100
800,0.240000
900,0.237300
1000,0.243700


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=4293, training_loss=0.16170730475304207, metrics={'train_runtime': 873.1976, 'train_samples_per_second': 78.659, 'train_steps_per_second': 4.916, 'total_flos': 416868952886508.0, 'train_loss': 0.16170730475304207, 'epoch': 3.0})

### metrics, Classification report

In [16]:
import numpy as np
from sklearn.metrics import classification_report

In [23]:
preds = trainer.predict(test_samples)
preds[:5]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


(array([[ 3.7610383, -4.441987 ],
        [ 3.8357818, -4.587597 ],
        [-2.5986586,  3.3775477],
        ...,
        [ 3.8078184, -4.480948 ],
        [-3.2292638,  4.039178 ],
        [ 3.3336968, -3.781662 ]], dtype=float32),
 array([0, 0, 1, ..., 0, 1, 1]),
 {'test_loss': 0.32059335708618164,
  'test_model_preparation_time': 0.0075,
  'test_runtime': 8.4284,
  'test_samples_per_second': 679.132,
  'test_steps_per_second': 42.475})

In [27]:
preds

PredictionOutput(predictions=array([[ 3.7610383, -4.441987 ],
       [ 3.8357818, -4.587597 ],
       [-2.5986586,  3.3775477],
       ...,
       [ 3.8078184, -4.480948 ],
       [-3.2292638,  4.039178 ],
       [ 3.3336968, -3.781662 ]], dtype=float32), label_ids=array([0, 0, 1, ..., 0, 1, 1]), metrics={'test_loss': 0.32059335708618164, 'test_model_preparation_time': 0.0075, 'test_runtime': 8.4284, 'test_samples_per_second': 679.132, 'test_steps_per_second': 42.475})

In [28]:
y_pred = np.argmax(preds.predictions, axis=1) 
y_true = [sample["labels"] for sample in test_samples]
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.95      0.93      2997
           1       0.94      0.91      0.93      2727

    accuracy                           0.93      5724
   macro avg       0.93      0.93      0.93      5724
weighted avg       0.93      0.93      0.93      5724



In [29]:
# we got an accuracy of 93%

## saving the model

In [30]:
trainer.save_model("./sarcasm_distilbert")
tokenizer.save_pretrained("./sarcasm_distilbert")

('./sarcasm_distilbert/tokenizer_config.json',
 './sarcasm_distilbert/special_tokens_map.json',
 './sarcasm_distilbert/vocab.txt',
 './sarcasm_distilbert/added_tokens.json',
 './sarcasm_distilbert/tokenizer.json')

## testing some headlines

In [37]:
def predict(headline):
        enc = tokenizer(
        headline,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt"
    )

        with torch.no_grad():
            outputs = model(**enc)
            probs = torch.softmax(outputs.logits, dim=1)[0]

        labels = ["Not Sarcastic", "Sarcastic"]
        pred_label = labels[probs.argmax().item()]
        confidence = probs.max().item()
    
        print("Prediction:", pred_label)
        print("Confidence:", confidence)

In [38]:
headline = "Nation Shudders At Large Block Of Uninterrupted Text" # test headline, expected sarcastic

predict(headline)

Prediction: Sarcastic
Confidence: 0.9992807507514954


In [41]:
headline = "Donald Trump says US will 'run' Venezuela and 'fix oil infrastructure'" # test headline, not sarcastic, typical Donald Trump

predict(headline)

Prediction: Not Sarcastic
Confidence: 0.9996745586395264
